In [2]:
%run ../credentials.py

In [3]:
import os 
from rich import print
from typing import List, Dict

import aocd
from aocd.models import Puzzle
DAY = 7
YEAR = 2022

os.environ['AOC_SESSION'] = SESION_ID
data = aocd.get_data(day = DAY, year = YEAR)

--- Day 7: No Space Left On Device ---

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:
```
$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
```

Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:
```
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
```
The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

    cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
        cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
        cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
        cd / switches the current directory to the outermost directory, /.
    ls means list. It prints out all of the files and directories immediately contained by the current directory:
        123 abc means that the current directory contains a file named abc with size 123.
        dir xyz means that the current directory contains a directory named xyz.

Given the commands and output in the example above, you can determine that the filesystem looks visually like this:
```
- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
```
Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:

    The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
    The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
    Directory d has total size 24933642.
    As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?



In [4]:
def set_value(dic :Dict, keys : List, value :str):
    for key in keys[:-1]:
        dic = dic.setdefault(key, {})
    dic[keys[-1]] = value



def state_dir(output :str, state_list: List, state_dct : Dict):
    """
    Keep track of the current directory and collect the list ouput into a dict
    dict_tree = {a : {b  : { e : 123,
                             f :124 },
         c :    {g : 324},

         d : 234
        ]

    state_list = ['/' 'to', 'file']
    
    }
    """
    match output.split():
        case ["cd", ".." ]: 
            if  1 < len(state_list):
                state_list.pop()
        case ["cd", new_state ]:
            state_list.append(new_state)
        case ["ls", *files ]:
            set_value(state_dct, state_list, return_tree(list_files= files))
        case _: 
            print("{_} not recognised!")
    return state_list, state_dct



def return_tree(list_files : List):
    subdir_tree = {}
    for i in range(0, len(list_files), 2):
        file_info = list_files[i]
        name = list_files[i + 1]
        if file_info == "dir":
            subdir_tree.get(name, [])
        else: 
            subdir_tree[name] = int(file_info)
    return subdir_tree



In [5]:
from  pprint import pprint
state = []
dir_tree = {}
for i in data.split('$ '):
    state_list, dir_tree = state_dir(output= i,
                                      state_list= state,
                                      state_dct= dir_tree)
pprint(dir_tree, indent= 4)

{_} not recognised!

{   '/': {   'fts': {   'dlqtffw': {'nqbvg.fgd': 73533},
                        'rbfmmjvd': {'zcgrgff.fnf': 290697},
                        'wvwhrb.dhh': 254713},
             'jnwr': {   'ghmtnzr': 323577,
                         'tdcbdpnr': 57588,
                         'wbv': {   'nmdwbnnr': {'scbcsb.pjg': 208370},
                                    'slzdbm.ncn': 253584},
                         'zzbvdcf': {'ssssrhwz': 8052}},
             'lrvl': {   'bqqltcg': {   'dlbjblbf': {'rnsrrf.rtv': 11732},
                                        'fdlw': {   'hlvpfw': {   'dhwq': {   'cgdpqh.tct': 127459,
                                                                              'jnwr.nqt': 58310}},
                                                    'mzsfcvgv.pqw': 228436},
                                        'jnwr': {   'ssssrhwz': 305998,
                                                    'vrt.qdt': 129135,
                                                    'wnvm.bld': 8

In [26]:
def calculate_directory_sizes(subdir, current_path, directory_sizes):
    if isinstance(subdir, dict):
        size = 0
        for key, value in subdir.items():
            path = current_path + '/' + key
            size += calculate_directory_sizes(value, path, directory_sizes)
        directory_sizes[current_path] = size
        return size
    elif isinstance(subdir, int):
        return subdir




def select_directories(dict_sizes, root = '/', upper_limit = 100000):
    directory_sizes = {}
    total_size = calculate_directory_sizes(dict_sizes[root], root, directory_sizes)
    total_limit_size = 0


    for directory, size in directory_sizes.items():
        if size < upper_limit:
            total_limit_size += size

    return total_limit_size, total_size, directory_sizes 


In [7]:

test_dict=  {   '/': {    'fts':     {   'dlqtffw':{'nqbvg.fgd': 73533},
                                          'rbfmmjvd': {
                                                       'zcgrgff.fnf': 290697},
                                                       'wvwhrb.dhh': 254713},
                            'jnwr':    {                 
                                                       'ghmtnzr': 323577,
                                                       'tdcbdpnr': 57588,
                            'wbv':     {   'nmdwbnnr':{
                                                       'scbcsb.pjg': 208370},
                                                       'slzdbm.ncn': 253584},
                            'zzbvdcf': {
                                          'ssssrhwz': 8052}}
}}



select_directories(dict_sizes = test_dict)

(81585, 1470114)

In [30]:
result_a, used_space, dir_dict = select_directories(dict_sizes = dir_tree)
aocd.submit(result_a, part="a", day=DAY, year=YEAR)

current_day is only available in December (EST)


aocd will not submit that answer again. At 2023-11-04 15:10:06.077446-04:00 you've previously submitted 1642503 and the server responded with:
That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


--- Part Two ---

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

In the example above, the total size of the outermost directory (and thus the total amount of used space) is 48381165; this means that the size of the unused space must currently be 21618835, which isn't quite the 30000000 required by the update. Therefore, the update still requires a directory with total size of at least 8381165 to be deleted before it can run.

To achieve this, you have the following options:
```
    Delete directory e, which would increase unused space by 584.
    Delete directory a, which would increase unused space by 94853.
    Delete directory d, which would increase unused space by 24933642.
    Delete directory /, which would increase unused space by 48381165.
```
Directories e and a are both too small; deleting them would not free up enough space. However, directories d and / are both big enough! Between these, choose the smallest: d, increasing unused space by 24933642.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?


In [33]:
space_needed = 40000000
new_limit = used_space - space_needed 
result_b = min([x for x in dir_dict.values() if x > new_limit])

In [34]:
aocd.submit(result_b, part="b", day=DAY, year=YEAR)

current_day is only available in December (EST)


Part b already solved with same answer: 6999588
